In [25]:
# !pip install numpy num2words nltk pandas Observations gensim
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln
from collections import Counter
from textblob import TextBlob

import operator
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

In [ ]:
import gensim
embeddings_index = gensim.models.KeyedVectors.load_word2vec_format('pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin', binary=True)

In [ ]:
stop_words = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [15]:
# embeddings_index = {}
# f = open('glove.6B.300d.txt')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

In [21]:
# dataset = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", header=None)
# dataset = dataset.head(1000)
# dataset[6] = preprocess(dataset[5])

In [22]:
dataset = pd.read_excel("OCR Output.xlsx")

In [112]:
def convert_numbers(k):
    for i in range(len(k)):
        try:
            num2words(int(k[i]))
            k[i] = " "
        except:
            pass
    return k

def get_cosine(a, b):
    return 1 - spatial.distance.cosine(a, b)

def preprocess(pd):
    pd = pd.apply(lambda x: str(TextBlob(x).correct()).decode('utf-8').strip())
    pd = pd.str.lower()
    pd = pd.str.replace('[{}]'.format('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n\t'), ' ')
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
    pd = pd.apply(lambda x: convert_numbers(x))
    pd = pd.str.join(' ')
    pd = pd.str.replace('[{}]'.format(string.punctuation), ' ')
    
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])    
    pd = pd.apply(lambda x: [item for item in x if item not in stop_words])
#     pd = pd.apply(lambda x: [stemmer.stem(y) for y in x])
    return pd

def processReviews(reviews, window=5, MAX_VOCAB_SIZE=1000):
#     vectorizer = SkipGramVectorizer(analyzer="word",stop_words="english",
#                                          max_features=MAX_VOCAB_SIZE,max_df=.75,min_df=10, k = window,ngram_range=(1,2))
    vectorizer = CountVectorizer(analyzer="word",tokenizer=None)
#                                       , preprocessor=None,stop_words="english",max_features=MAX_VOCAB_SIZE,max_df=.5,min_df=5)
    train_data_features = vectorizer.fit_transform(reviews)
    words = vectorizer.get_feature_names()
    vocabulary = dict(zip(words,np.arange(len(words))))
    inv_vocabulary = dict(zip(np.arange(len(words)),words))
    wordOccurenceMatrix = train_data_features.toarray()
    return wordOccurenceMatrix, vocabulary, words

In [ ]:
dataset[6] = preprocess(dataset['Text'])

In [28]:
df = {}
for i in dataset[6]:
    for word in Counter(i).keys():
        try:
            df[word] += 1
        except:
            df[word] = 1

In [29]:
sorted_x = sorted(df.items(), key=operator.itemgetter(1), reverse=True)

In [30]:
high_df = [i[0] for i in sorted_x[:200]]
dataset[7] = dataset[6].apply(lambda x: " ".join(x))

In [31]:
def deep_clean(data):
    text = ""
    for i in data.split(" "):
        if i not in high_df and len(i)>2:
            text += i +" "
        else:
            pass
    return text.rstrip()

In [32]:
dataset[7] = dataset[7].apply(lambda x: deep_clean(x))

In [33]:
matrix, vocabulary, words = processReviews(dataset[7].values)

In [34]:
docs_edges = []
count = 0
ignored = []
taken = []
for idx, doc in enumerate(dataset[6].values):
    edges = []
    for i in doc:
        for j in doc:
            if i != j:
                try:
                    a = embeddings_index[i]
                    b = embeddings_index[j]
                    if get_cosine(a, b) > 0.5:
                        edges.append((vocabulary[i], vocabulary[j]))
                except:
                    try:
                        embeddings_index[i]
                        taken.append(i)
                    except:
                        ignored.append(i)
                    try:
                        embeddings_index[j]
                    except:
                        ignored.append(j)
                        taken.append(j)
                    pass
    docs_edges.append(edges)
    
    if idx % 10 == 0:
        print(idx)

0
10
20
30
40
50
60
70
80
90


In [47]:
# !pip install -U textblob
# !python -m textblob.download_corpora

In [11]:
edge_dict = {}
for i in docs_edges:
    for j in i:
#         print(j)
        try:
            edge_dict[j[0]] += [j[1]]
        except:
            edge_dict[j[0]] = [j[1]]

In [12]:
import imp

In [13]:
import lda2

In [14]:
lambda_param = 0.8
N_TOPICS = 10

In [15]:
imp.reload(lda2)
sampler = lda2.LdaSampler(n_topics=N_TOPICS, lambda_param=lambda_param)

In [16]:
for it, phi in enumerate(sampler.run(matrix, edge_dict)):
    print "Iteration", it
    print "Likelihood", sampler.loglikelihood(docs_edges)
    for i in (sampler.getTopKWords(5, words)):
        print(sampler.getTopKWords(5, words)[i])

Iteration 0
Likelihood -71731.7146030965
[u'today', u'realli', u'like', u'miss', u'wa']
[u'home', u'one', u'get', u'miss', u'sad']
[u'go', u'much', u'show', u'see', u'tomorrow']
[u'quot', u'sick', u'one', u'work', u'wa']
[u'gt', u'one', u'http', u'work', u'want']
[u'quot', u'well', u'realli', u'today', u'tonight']
[u'earli', u'know', u'get', u'work', u'bed']
[u'alreadi', u'peopl', u'good', u'day', u'sad']
[u'like', u'need', u'miss', u'get', u'time']
[u'good', u'realli', u'bad', u'work', u'sleep']
Iteration 1
Likelihood -68751.07829038508
[u'miss', u'wish', u'watch', u'im', u'realli']
[u'good', u'go', u'sad', u'miss', u'day']
[u'tri', u'miss', u'sorri', u'go', u'like']
[u'world', u'much', u'sad', u'work', u'get']
[u'work', u'get', u'go', u'know', u'one']
[u'like', u'today', u'tonight', u'well', u'work']
[u'still', u'bed', u'cri', u'quot', u'work']
[u'hate', u'home', u'wa', u'got', u'good']
[u'better', u'like', u'time', u'miss', u'get']
[u'night', u'find', u'could', u'hurt', u'sleep']
It

Iteration 17
Likelihood -64432.636504315364
[u'time', u'sorri', u'hear', u'http', u'com']
[u'look', u'wish', u'lt', u'watch', u'miss']
[u'like', u'feel', u'see', u'tomorrow', u'hate']
[u'work', u'sorri', u'sad', u'one', u'wa']
[u'need', u'oh', u'know', u'think', u'get']
[u'feel', u'come', u'hour', u'work', u'night']
[u'read', u'found', u'poor', u'quot', u'miss']
[u'meet', u'news', u'whole', u'watch', u'good']
[u'twitter', u'get', u'com', u'miss', u'http']
[u'get', u'still', u'find', u'today', u'realli']
Iteration 18
Likelihood -64492.12533592274
[u'hear', u'com', u'cri', u'http', u'wa']
[u'start', u'morn', u'lt', u'watch', u'miss']
[u'tomorrow', u'feel', u'hope', u'see', u'hate']
[u'wa', u'sorri', u'sad', u'get', u'time']
[u'need', u'know', u'get', u'oh', u'think']
[u'night', u'hour', u'feel', u'work', u'sleep']
[u'poor', u'new', u'go', u'miss', u'quot']
[u'meet', u'new', u'make', u'watch', u'good']
[u'twitter', u'get', u'miss', u'com', u'http']
[u'still', u'today', u'get', u'find', u'